<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/Test_and_Trash/ReHealthCo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Löschen aller Variablen
%reset -f

In [ ]:
# import libraries
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
!pip install ydata_profiling
# from functools import reduce - benötigt?

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
# mount drive to access data
# from google.colab import drive
# drive.mount("/content/drive")

## import from one Google colab notebook to the other

In [ ]:
# !pip install import_ipynb
# !pip install ipynb

# import import_ipynb
# import ipynb

In [ ]:
# import ReHealthCo_Rohdatenbank

# Github-Repositorium im Colab-Umgebung clonen

In [ ]:
!git clone https://github.com/Fuenfgeld/DMA2023TeamD

In [ ]:
# %cd /content/DMA2023TeamD

In [ ]:
# !ls

In [ ]:
# Connect to source database
def create_connection(db_file):
  """ create a database connection to SQLite database"""
  conn = None
  try:
    conn = sqlite3.connect(db_file)
    return conn
  except Error as error:
    print("Error while connecting to sqlite", error)
  # finally:
  #   if conn:
  #     conn.close()

# Erstellung der (leeren) Datenbanken (asthma, covid, metabolic, rehealthco)

In [ ]:
material_path = "/content/DMA2023TeamD/"
# conn = create_connection() - verbindung zur Datenbank
# cur = conn.cursor() - objekt auf datenbank, das die SQL-Anfragen ausführen kann

In [ ]:
# create a temporary empty ast_database in the memory
ast_db = f"{material_path}/source_dbs/asthma.db"
ast_conn = create_connection(ast_db)
ast_cur = ast_conn.cursor()
ast_conn.commit()
print("Database connection is successfully")

In [ ]:
# create a temporary empty cov_database in the memory
cov_db = f"{material_path}/source_dbs/covid.db"
cov_conn = create_connection(cov_db)
cov_cur = cov_conn.cursor()
cov_conn.commit()
print("Database connection is successfully")

In [ ]:
# create a temporary empty met_database in the memory
met_db = f"{material_path}/source_dbs/metabolic.db"
met_conn = create_connection(met_db)
met_cur = met_conn.cursor()
met_conn.commit()
print("Database connection is successfully")

In [ ]:
# create a temporary empty rhc_database in the memory
rhc_db = f"{material_path}/source_dbs/rehealthco.db"
rhc_conn = create_connection(rhc_db)
rhc_cur = rhc_conn.cursor()
rhc_conn.commit()
print("Database connection is successfully")

# Erstellung der Tabellen

In [ ]:
path = f"{material_path}/source_dbs/sql_create.txt"
myfile = ""
with open(path, "r") as f:
  myfile=f.read()
#  print(myfile)

In [ ]:
ast_cur.executescript(myfile)
ast_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in ast_database
ast_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(ast_cur.fetchall())

In [ ]:
cov_cur.executescript(myfile)
cov_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in cov_database
# cov_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cov_cur.fetchall())

In [ ]:
met_cur.executescript(myfile)
met_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in met_database
met_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(met_cur.fetchall())

In [ ]:
rhc_cur.executescript(myfile)
rhc_conn.commit()
print("Successfully created tables in the database")

In [ ]:
# List of tables in cov_database
# rhc_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(rhc_cur.fetchall())

# Datenbanken befüllen mit csv-Dateien

In [ ]:
csv_path_ast = f"{material_path}csv_data/asthma"
csv_path_cov = f"{material_path}csv_data/covid19"
csv_path_met = f"{material_path}csv_data/metabolic_syndrome_disease"

In [ ]:
# load the data into a Pandas DataFrame

astpat_df = pd.read_csv(f"{csv_path_ast}/patients.csv")
astcon_df = pd.read_csv(f"{csv_path_ast}/conditions.csv")
astmed_df = pd.read_csv(f"{csv_path_ast}/medications.csv")
astenc_df = pd.read_csv(f"{csv_path_ast}/encounters.csv")

covpat_df = pd.read_csv(f"{csv_path_cov}/patients.csv")
covcon_df = pd.read_csv(f"{csv_path_cov}/conditions.csv")
covmed_df = pd.read_csv(f"{csv_path_cov}/medications.csv")
covenc_df = pd.read_csv(f"{csv_path_cov}/encounters.csv")

metpat_df = pd.read_csv(f"{csv_path_met}/patients.csv")
metcon_df = pd.read_csv(f"{csv_path_met}/conditions.csv")
metmed_df = pd.read_csv(f"{csv_path_met}/medications.csv")
metenc_df = pd.read_csv(f"{csv_path_met}/encounters.csv")

# Abschluss
ast_conn.commit()
cov_conn.commit()
met_conn.commit()

In [ ]:
# Hinzufügen einer zusätzlichen Spalte mit dem Wert asthma/covid/metabolic; CAVE: späterer Copy-table Befehl ist kein df sondern ließt Ursprungstabelle, daher ggf. Tabellendatenbankart mit DEFAULT wert schon in txt-Datei einfügen
astpat_df['datenbankart'] = 'asthma'
astcon_df['datenbankart'] = 'asthma'
astmed_df['datenbankart'] = 'asthma'
astenc_df['datenbankart'] = 'asthma'

covpat_df['datenbankart'] = 'covid'
covcon_df['datenbankart'] = 'covid'
covmed_df['datenbankart'] = 'covid'
covenc_df['datenbankart'] = 'covid'

metpat_df['datenbankart'] = 'metabolic'
metcon_df['datenbankart'] = 'metabolic'
metmed_df['datenbankart'] = 'metabolic'
metenc_df['datenbankart'] = 'metabolic'

# Abschluss
ast_conn.commit()
cov_conn.commit()
met_conn.commit()

In [ ]:
# write the data to a sqlite table
astpat_df.to_sql("patients", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astcon_df.to_sql("conditions", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astmed_df.to_sql("medications", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
astenc_df.to_sql("encounters", ast_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

covpat_df.to_sql("patients", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covcon_df.to_sql("conditions", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covmed_df.to_sql("medications", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
covenc_df.to_sql("encounters", cov_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

metpat_df.to_sql("patients", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metcon_df.to_sql("conditions", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metmed_df.to_sql("medications", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
metenc_df.to_sql("encounters", met_conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

# Abschluss
ast_conn.commit()
cov_conn.commit()
met_conn.commit()

In [ ]:
# check dataframe asthma/covid19/metabolic
# print(astpat_df)
# print(astcon_df)
# print(astmed_df)
# print(astenc_df)

# print(covpat_df)
# print(covcon_df)
# print(covmed_df)
# print(covpro_df)

# print(metpat_df)
# print(metcon_df)
# print(metmed_df)
# print(metpro_df)

In [ ]:
# Check data in patients

# check data in a table astpat
# ast_cur.execute("SELECT * FROM patients;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covpat
# cov_cur.execute("SELECT * FROM patients;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metpat
# met_cur.execute("SELECT * FROM patients;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in conditions

# check data in a table astcon
# ast_cur.execute("SELECT * FROM conditions;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covcon
# cov_cur.execute("SELECT * FROM conditions;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metcon
# met_cur.execute("SELECT * FROM conditions;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in medications

# check data in a table astmed
# ast_cur.execute("SELECT * FROM medications;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covmed
# cov_cur.execute("SELECT * FROM medications;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metmed
# met_cur.execute("SELECT * FROM medications;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# Check data in encounters

# check data in a table astpro
# ast_cur.execute("SELECT * FROM encounters;")
# rows = ast_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table covpro
# cov_cur.execute("SELECT * FROM encounters;")
# rows = cov_cur.fetchall()
# for row in rows:
#  print(row)

# check data in a table metpro
# met_cur.execute("SELECT * FROM encounters;")
# rows = met_cur.fetchall()
# for row in rows:
#  print(row)

# Copy a table from one database to another
(empty Rehealthco SQLite database schon erstellt)

In [ ]:
# Get connections to the databases 
# von asthma nach rehealthco
def copy_table(rhc_conn, ast_cur, rhc_cur, table_name):
  ast_cur.execute(f'SELECT * FROM {table_name}')
  output = ast_cur.fetchall()   # Returns the results as a list.
  # Insert those contents into another table.
  for row in output:
      rowlen = len(row)
      repstring = ""
      for i in range(rowlen -1):
        repstring += "?,"
      repstring += "?"
      rhc_cur.execute(f'INSERT INTO {table_name} VALUES ({repstring})', row)
  # Cleanup
  rhc_conn.commit()

In [ ]:
# alle Tabellen vereinen in rehealthco
copy_table(rhc_conn, ast_cur, rhc_cur, "patients") # 1035 patienten
copy_table(rhc_conn, cov_cur, rhc_cur, "patients") # nun 2085 patienten
copy_table(rhc_conn, met_cur, rhc_cur, "patients") # nun 3113 patienten

copy_table(rhc_conn, ast_cur, rhc_cur, "conditions") # 260 conditions
copy_table(rhc_conn, cov_cur, rhc_cur, "conditions") # 6895 conditions
copy_table(rhc_conn, met_cur, rhc_cur, "conditions") # 13733 conditions

copy_table(rhc_conn, ast_cur, rhc_cur, "medications") # 1515 medications
copy_table(rhc_conn, cov_cur, rhc_cur, "medications") # 2910 medications
copy_table(rhc_conn, met_cur, rhc_cur, "medications") # 4625 medications

copy_table(rhc_conn, ast_cur, rhc_cur, "encounters") # 8522 encounters
copy_table(rhc_conn, cov_cur, rhc_cur, "encounters") # 35368 encounters
copy_table(rhc_conn, met_cur, rhc_cur, "encounters") # 63081 encounters


In [ ]:
# Tabelleninhalt rehealthco als pandas dataframe
rhcpat_df = pd.read_sql('''SELECT * FROM patients''', rhc_conn)
rhccon_df = pd.read_sql('''SELECT * FROM conditions''', rhc_conn)
rhcmed_df = pd.read_sql('''SELECT * FROM medications''', rhc_conn)
rhcenc_df = pd.read_sql('''SELECT * FROM encounters''', rhc_conn)

# print(rhcpat_df)
# print(rhccon_df)
# print(rhcmed_df)
# print(rhcenc_df)

In [ ]:
# Kompelette Tabelle abfragen

# rhc_cur.execute("SELECT * FROM patients;")
# rows = rhc_cur.fetchall()
# for row in rows:
#  print(row)

# rhc_cur.execute("SELECT * FROM conditions;")
# rows = rhc_cur.fetchall()
# for row in rows:
#  print(row)

# rhc_cur.execute("SELECT * FROM medications;")
# rows = rhc_cur.fetchall()
# for row in rows:
#  print(row)

# rhc_cur.execute("SELECT * FROM encounters;")
# rows = rhc_cur.fetchall()
# for row in rows:
#  print(row)

In [ ]:
# 1-Zeiltenanfrage
# rhc_cur.execute("SELECT * FROM conditions")
# out = rhc_cur.fetchall()
# print(out)

# Indexe Rehealthco

In [ ]:
#funktion "f" bestimmmte Datenbank -> sql-anfrage
def f(cursor, sql_query):
  cursor.execute(sql_query)
  cursor.fetchall()
  print(cursor.fetchall())

In [ ]:
f(rhc_cur, "CREATE INDEX idx_contacts_name ON patients (FIRST, LAST)")
# statt multicolumn Index
# rhc_cur.execute("CREATE INDEX idx_contacts_name ON patients (FIRST, LAST)")
# rhc_cur.fetchall()

In [ ]:
# multicolumn Index
# rhc_cur.execute("CREATE INDEX idx_contacts_name ON patients (FIRST, LAST)")
# rhc_cur.fetchall()

In [ ]:
# show list indexes created for a table
rhc_cur.execute("PRAGMA index_list(patients)")
print(rhc_cur.fetchall())

In [ ]:
rhc_cur.execute("CREATE INDEX idx_patname_conditions ON conditions (PATIENT)")
rhc_cur.execute("PRAGMA index_list(conditions)")
print(rhc_cur.fetchall())

In [ ]:
rhc_cur.execute("CREATE INDEX idx_patname_medications ON medications (PATIENT)")
rhc_cur.execute("PRAGMA index_list(medications)")
print(rhc_cur.fetchall())

In [ ]:
rhc_cur.execute("CREATE INDEX idx_patname_encounters ON encounters (PATIENT)")
rhc_cur.execute("PRAGMA index_list(encounters)")
print(rhc_cur.fetchall())

In [ ]:
# get all indexes from a database 
rhc_cur.execute("select type, name, tbl_name, sql FROM sqlite_master WHERE type='index'")
print(rhc_cur.fetchall())

In [ ]:
# # Drop index
# # cur.execute("DROP INDEX idx_contacts_name")
# cur.execute("PRAGMA index_list(patients)")
# print(cur.fetchall())

# Query the tables, join two tables - alle Tabellen in rhc_vereinall_df vereinen

In [ ]:
# verbindung_astpat_astmed = "SELECT * FROM medications INNER JOIN patients ON medications.PATIENT = patients.Id"
# ast_cur.execute(verbindung_astpat_astmed)
# records = ast_cur.fetchall()
# for row in records:
#   print(row)

In [ ]:
# Left Join
# cov_cur.execute('''SELECT * FROM medications LEFT JOIN patients ON medications.PATIENT = patients.Id''')
# records = cov_cur.fetchall()
# for row in records:
#   print(row)

In [ ]:
# alternativ über pandas:
# print(rhcpat_df)
# print(rhccon_df)
# print(rhcmed_df)
# print(rhcenc_df)

# Beispiel:
# df = pd.DataFrame.merge(df, df_procedures, left_on="Id", right_on="PATIENT", how="inner")
# df.drop("PATIENT", axis=1, inplace=True)
# df.shape

In [ ]:
# show all columns
pd.set_option('max_columns', None)
# show all rows
# pd.set_option('display.max_rows', None)
# show ten rows
# pd.set_option('display.max_rows', 10)

In [ ]:
rhc_verein1_df = pd.DataFrame.merge(rhcpat_df, rhccon_df, left_on=["Id", "datenbankart"], right_on=["PATIENT", "datenbankart"], how="inner")
rhc_verein1_df.drop("PATIENT", axis=1, inplace=True)
rhc_verein1_df.rename(columns={"DESCRIPTION": "desc_con", "CODE": "code_con", "START": "start_con", "STOP": "stop_con"}, inplace=True)
# rhc_verein1_df.shape
# rhc_verein1_df.head()
print(rhc_verein1_df)

In [ ]:
#@title Default title text { display-mode: "both" }
rhc_verein2_df = pd.DataFrame.merge(rhc_verein1_df, rhcmed_df, left_on=["Id", "datenbankart", "ENCOUNTER"], right_on=["PATIENT", "datenbankart", "ENCOUNTER"], how="inner")
rhc_verein2_df.drop("PATIENT", axis=1, inplace=True)
rhc_verein2_df.rename(columns={"DESCRIPTION": "desc_med", "CODE": "code_med", "START": "start_med", "STOP": "stop_med", "REASONDESCRIPTION": "reasondesc_med", "REASONCODE": "reasoncode_med"}, inplace=True)
rhc_verein2_df.shape
# rhc_verein2_df.head()
# print(rhc_verein2_df)

In [ ]:
rhcenc_df.rename(columns={"Id": "id_enc"}, inplace=True)
# print(rhcenc_df)

In [ ]:
rhc_vereinall_df = pd.DataFrame.merge(rhc_verein2_df, rhcenc_df, left_on=["Id", "datenbankart", "PAYER", "PAYER_COVERAGE"], right_on=["PATIENT", "datenbankart", "PAYER", "PAYER_COVERAGE"], how="inner")
rhc_vereinall_df.drop("PATIENT", axis=1, inplace=True)
rhc_vereinall_df.rename(columns={"DESCRIPTION": "desc_enc", "CODE": "code_enc", "START": "start_enc", "STOP": "stop_enc", "REASONDESCRIPTION": "reasondesc_enc", "REASONCODE": "reasoncode_enc"}, inplace=True)
rhc_vereinall_df.shape
# rhc_vereinall_df.head()

In [ ]:
  # Cleanup
  rhc_conn.commit()

In [ ]:
rhc_vereinall_df_to_csv = pd.DataFrame(rhc_vereinall_df)
rhc_vereinall_df.to_csv(index=False)
print(rhc_vereinall_df.columns.tolist())

In [ ]:
# pandas to csv
# df = pd.DataFrame({'name': ['Raphael', 'Donatello'],
#                    'mask': ['red', 'purple'],
#                    'weapon': ['sai', 'bo staff']})
# df.to_csv(index=False)

In [ ]:
from pathlib import Path  
filepath = Path('/content/drive/Shareddrives/Projektarbeit DMA Gruppe D/out.csv') # gdrive-sharefolder
# filepath = Path(f"{material_path}/source_dbs/out.csv") # github-Folder
filepath.parent.mkdir(parents=True, exist_ok=True)  
rhc_vereinall_df.to_csv(filepath)  

In [ ]:
# To write a csv file to a new folder or nested folder you will first need to create it using Pathlib 
# from pathlib import Path  
# filepath = Path('folder/subfolder/out.csv')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# df.to_csv(filepath)  

In [ ]:
# rename / Umbennen variables
# df_procedures.rename(columns={"CODE":"CODE_PROCEDURE", "DESCRIPTION":"PROCEDURE_DESCRIPTION"}, inplace=True)
# df_procedures = df_procedures[["PATIENT", "CODE_PROCEDURE", "PROCEDURE_DESCRIPTION"]]

In [ ]:
# columns returns the name of all of your columns in the dataset.
# rhcpat_df.columns

In [ ]:
# Removing Duplicate Rows
# print("Number of Duplicated Rows", df.duplicated(df.columns).sum())

In [ ]:
# dropping duplicate values if exists and keep first one
# rhcpat_df.drop_duplicates(keep="first",inplace=True) 
# print("Size of dataset after removinf duplicated rows", rhcpat_df.shape)

# Prediction (Supervised Learning)

In [ ]:
# install the newest version,  
# uncomment this line in first runtime, it needs to restart runtime
!pip3 install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

# Exploratory Data Analysis (EDA)


In [ ]:
rhc_vereinall_df.shape

In [ ]:
rhc_vereinall_df

In [ ]:
# ydata-profiling(rhc_vereinall_df)
ProfileReport(rhc_vereinall_df)

In [ ]:
ProfileReport(rhcpat_df) # deathdate: 3000 fehlende Todesdaten, bei insg. 3113 Patienten -> 113 Tote

In [ ]:
# ProfileReport(rhccon_df) # 43 unterschiedliche Descriptions
# ProfileReport(rhcmed_df)
ProfileReport(rhcenc_df)

In [ ]:
#Übersicht der Condition-Kohorte also insgesamt 43 verschiedene conditions
rhccon_df.nunique(axis=0)

In [ ]:
# rhccon_df.columns
# number of patients in procedure table
rhccon_df.groupby(["DESCRIPTION"]).size()
# rhccon_df.groupby(["PATIENT", "DESCRIPTION"]).size() # sortiert nach 2 Spalten

In [ ]:
# Löschen bestimmter Spaltenwerte - alles von bis zu History of cardiac arrest (situation) gelöscht 
# df.drop(df[df['Fee'] >= 24000].index, inplace = True)
dropquatschdescription_df = rhccon_df.drop(rhccon_df[ (rhccon_df['DESCRIPTION'] == "Wheezing (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Vomiting symptom (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Suspected COVID-19") |
                                                     (rhccon_df['DESCRIPTION'] == "Sputum finding (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Sore throat symptom (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Respiratory distress (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Passive conjunctival congestion (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Nausea (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Nasal congestion (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Muscle pain (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Loss of taste (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Joint pain (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "History of myocardial infarction (situation)") |
                                                     (rhccon_df['DESCRIPTION'] == "History of cardiac arrest (situation)") |
                                                     (rhccon_df['DESCRIPTION'] == "Hemoptysis (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Headache (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Fever (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Fatigue (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Dyspnea (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Diarrhea symptom (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Cough (finding)") |
                                                     (rhccon_df['DESCRIPTION'] == "Chill (finding)")
                                                     ].index)

In [ ]:
# oder #Zunächst Selektion bestimmter Spaltenwerte
# auswahl_nur_pos_cov_df = rhccon_df.loc[(rhccon_df["CODE"] == "94531-1") & (rhccon_df["VALUE"] == "Detected (qualifier value)")] 
# auswahl_nur_pos_cov_description_df.head(3)

In [ ]:
# Zusammenfügen von 2 Dataframes in einen Dataframe
# pos_plus_neg_cov_df = pd.concat([allerg_postest_df, wirklich_nur_neg_cov_df])
# pos_plus_neg_cov_df.head(3)

In [ ]:
dropquatschdescription_df.nunique(axis=0)

In [ ]:
dropquatschdescription_df.groupby(["PATIENT", "DESCRIPTION"]).size()

In [ ]:
# # convert categorical variable to numerical by replacing - könnte wichtig für die Erstellung von Histogrammen sein! Beispiel
# df.GENDER.replace(['M', 'F'], [0, 1], inplace=True)
# df.RACE.replace(["white", "black", "asian"], [1, 2, 3], inplace=True)

In [ ]:
dropquatschdescription_df.groupby(["PATIENT"]).size()

In [ ]:
# Anzeige fehlender Werte, z.B. fehlender Todestag, hier 3000
# rhcpat_df.isnull().sum() # alle fehlenden Werte
# rhcpat_df.DEATHDATE.isnull().sum() # eine bestimmte Spalte

In [ ]:
# rhcpat_df.BIRTHDATE.shape # Anzeige wieviel Zeilen eine Spalte besitzt
# rhcpatbirth_df = rhcpat_df[["Id", "BIRTHDATE"]]
# rhcpatbirth_df.shape # Anzeige aller Zeilen zweier Spalten

In [ ]:
# calculate age 
# fill nall values with todays date
rhcpat_df["DEATHDATE"] = rhcpat_df.DEATHDATE.fillna(pd.to_datetime("today"))
# convert to datetime
rhcpat_df["DEATHDATE"] = pd.to_datetime(rhcpat_df["DEATHDATE"])
rhcpat_df["BIRTHDATE"] = pd.to_datetime(rhcpat_df["BIRTHDATE"])
rhcpat_df["AGE"] = rhcpat_df.DEATHDATE.dt.year - rhcpat_df.BIRTHDATE.dt.year
# select variables
rhcpatage_df = rhcpat_df[["Id", "AGE", "GENDER", "HEALTHCARE_EXPENSES"]]
rhcpatage_df.head()
ProfileReport(rhcpatage_df)

In [ ]:
rhcpatage_df

In [ ]:
# Histogramm
# rhcpat_df["AGE"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

# Boxplot and Remove Outlier 
(müsste das Extremwerte rausnehmen sein)

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(x=rhcpat_df["AGE"])

In [ ]:
rhcpatageunterhunter_df = rhcpat_df[rhcpat_df["AGE"]<100]
# rhcpat_df.shape
sns.boxplot(x=rhcpatageunterhunter_df["AGE"])

In [ ]:
# distribution of GENDER attribute
sns.countplot(x="variable", hue= "value", data=pd.melt(rhcpat_df[["GENDER"]]))

In [ ]:
# distribution of DESCRIPTION attribute
# sns.countplot(x="variable", hue= "value", data=pd.melt(rhccon_df[["DESCRIPTION"]]))
sns.countplot(x="variable", hue= "value", data=pd.melt(dropquatschdescription_df[["DESCRIPTION"]]))

In [ ]:
dropquatschdescription_df.groupby(["PATIENT", "DESCRIPTION"]).size()

In [ ]:
dropquatschdescription_df.groupby(["DESCRIPTION"]).size()

In [ ]:
sns.countplot(x="variable", hue= "value", data=pd.melt(dropquatschdescription_df[["PATIENT"]]))

In [ ]:
# # convert categorical variable to numerical by replacing - könnte wichtig für die Erstellung von Histogrammen sein! Beispiel
# df.GENDER.replace(['M', 'F'], [0, 1], inplace=True)
# df.RACE.replace(["white", "black", "asian"], [1, 2, 3], inplace=True)

In [ ]:
# HEALTHCARE_EXPENSES uncovered
rhcpat_df["Verhaeltnis0"] = rhcpat_df["HEALTHCARE_EXPENSES"]-rhcpat_df["HEALTHCARE_COVERAGE"]
# print(rhcpat_df["Verhaeltnis0"])

In [ ]:
# Histogramm Verhätnis1
rhcpat_df["Verhaeltnis0"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
# Verhältnis HEALTHCARE_EXPENSES im Verhältnis mit HEALTHCARE_COVERAGE
rhcpat_df["Verhaeltnis1"] = rhcpat_df["HEALTHCARE_COVERAGE"]/rhcpat_df["HEALTHCARE_EXPENSES"]
# print(rhcpat_df["Verhaeltnis1"])

In [ ]:
# Histogramm Verhätnis1
rhcpat_df["Verhaeltnis1"].plot(kind='hist', bins=20, figsize=(12,6), facecolor='grey',edgecolor='black')

In [ ]:
ProfileReport(rhcpat_df)

# Multivariate Analysis: Analyzing Relationships Between Variables

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Generating the correlating matrix
corr = rhcpat_df.corr()
plt.figure(figsize=(7, 5))
# Generating the correlation heat-map
sns.heatmap(corr, annot=True)

In [ ]:
import numpy as np

In [ ]:
# funktioniert noch nicht - benötigt?
# # select feature with correlation less than a threshold
# columns = np.full((corr.shape[0],), True, dtype=bool)
# for i in range(corr.shape[0]):
#     for j in range(i+1, corr.shape[0]):
#         if corr.iloc[i,j] >= 0.7:
#             if columns[j]:
#                 columns[j] = False

# selected_columns = numdf.columns[columns]
# selected_columns

# Scatterplot

In [ ]:
rhcpat_df.plot(kind='scatter', y='HEALTHCARE_EXPENSES', x='BIRTHDATE')

In [ ]:
# sklearn aus beastcancer notebook
from sklearn.model_selection import train_test_split
from sklearn import tree
import sklearn.metrics
import graphviz

!pip install dtreeviz
from dtreeviz import * 

In [ ]:
# Generating the correlating matrix
corr = rhcpat_df.corr()
plt.figure(figsize=(30, 15))
# Generating the correlation heat-map
sns.heatmap(corr, annot=True)

In [ ]:
# meansDF = cancerDF[['mean radius',
#  'mean texture',
#  'mean perimeter',
#  'mean area',
#  'mean smoothness',
#  'mean compactness',
#  'mean concavity',
#  'mean concave points',
#  'mean symmetry',
#  'mean fractal dimension',
#  'target']]

In [ ]:
# sns.set_theme(style="ticks")

# sns.pairplot(meansDF, hue="target")

In [ ]:
# X = cancerDF[['mean radius',
#  'mean texture',
#  'mean perimeter',
#  'mean area',
#  'mean smoothness',
#  'mean compactness',
#  'mean concavity',
#  'mean concave points',
#  'mean symmetry',
#  'mean fractal dimension',
#  'radius error',
# 'texture error',
#  'perimeter error',
#  'area error',
#  'smoothness error',
#  'compactness error',
#  'concavity error',
#  'concave points error',
#  'symmetry error',
#  'fractal dimension error',
#  'worst radius',
#  'worst texture',
#  'worst perimeter',
#  'worst area',
#  'worst smoothness',
#  'worst compactness',
#  'worst concavity',
#  'worst concave points',
#  'worst symmetry',
#  'worst fractal dimension']]
# 
# Y = cancerDF[['target']]

In [ ]:
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0) # Random state auf 0 setzen wegen der Reproduzierbarkeit!

In [ ]:
# weiter mit Decision Tree

# Prediction

In [ ]:
# import python packages for ML models
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

from sklearn.model_selection import  GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
# rhcpatage_df age + lifetimecost (healthcare_expenses)
rhcpatage_df.head()

In [ ]:
dfpred = rhcpatage_df.pivot_table(values="HEALTHCARE_EXPENSES", index=None, columns=["AGE", "GENDER"], aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)
dfpred.head(20)
# dfpred.shape

In [ ]:
dfpred = rhcpatage_df

In [ ]:
rhcpatage_df[["AGE", "GENDER", "HEALTHCARE_EXPENSES"]].groupby(["AGE", "GENDER"]).mean().reset_index()

In [ ]:
dfpred = rhcpatage_df[["AGE", "GENDER", "HEALTHCARE_EXPENSES"]].groupby(["AGE", "GENDER"]).mean().reset_index()

In [ ]:
type(dfpred)

In [ ]:
# dfpred.groupby(["Id"]).size()

In [ ]:
# Mischen der Daten

dfpred= dfpred.sample(frac=1)

In [ ]:
# Splitten der Daten in train und test Dataframes
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

In [ ]:
dfpred

In [ ]:
# Ausgabe der Shapes von train und test Dataframes
dfpred_train, dfpred_test = train_test_split(dfpred, test_size=test_ratio, random_state=0)
dfpred_train.shape, dfpred_test.shape

In [ ]:
print(dfpred_train)

In [ ]:
# Bestimmen der Features und des Zielwertes
X = dfpred_train["AGE"] # ggf. [["AGE", "GENDER"]] und Gender als Zahl
# X = pd.DataFrame(rhcpatage_df["AGE"])
y = dfpred_train["HEALTHCARE_EXPENSES"]
# y = pd.DataFrame(rhcpatage_df["HEALTHCARE_EXPENSES"])

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
y.head()

In [ ]:
y.shape

In [ ]:
pred_test_df = pd.concat([X, y], axis=1)
pred_test_df.shape

In [ ]:
# shuffle
pred_test_df.sample(frac=1)

In [ ]:
X = pred_test_df.drop("HEALTHCARE_EXPENSES", axis=1)
y = pred_test_df["HEALTHCARE_EXPENSES"]

In [ ]:
# split data to testing and training dataset
from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # ACHTUNG 42!!
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# import regression model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
# create an object
linreg = LinearRegression()

In [ ]:
ElasticNet = ElasticNet()
SGDRegressor = SGDRegressor()

In [ ]:
RF = RandomForestRegressor()

In [ ]:
RF.fit(X_train, y_train)

In [ ]:
ElasticNet.fit(X_train, y_train)

In [ ]:
test_pred_elasticnet = ElasticNet.predict(X_test)
test_pred_elasticnet

In [ ]:
SGDRegressor.fit(X_train, y_train)

In [ ]:
test_pred_SGD = SGDRegressor.predict(X_test)
test_pred_SGD

In [ ]:
# train the model
linreg.fit(X_train, y_train)

In [ ]:
neutest_pred = linreg.predict(pd.DataFrame({"AGE":[40]}))
neutest_pred

In [ ]:

# Making prediction using testing set
test_pred = linreg.predict(X_test)
test_pred

In [ ]:
linreg.score(X_test, y_test) #x groß da Dataframe/Matrix und y klein weil Liste/Vektor

In [ ]:
ElasticNet.score(X_test, y_test) 

In [ ]:
SGDRegressor.score(X_test, y_test) 

In [ ]:
RF.score(X_test, y_test) 

# Colab Book reproduzierbar machen

In [ ]:
# from sklearn.model_selection import train_test_split
import sklearn.metrics

import matplotlib
import matplotlib.pyplot as plt

## Configuration

In [ ]:
%matplotlib inline

# Version Check

In [ ]:
#python Version
import sys
sys.version_info

In [ ]:
pd.__version__

In [ ]:
%pip install pandas

In [ ]:
sns.__version__

In [ ]:
matplotlib.__version__

In [ ]:
np.__version__

In [ ]:
sklearn.__version__

In [ ]:
graphviz.__version__


In [ ]:
%pip freeze

# Calculate the Checksum of the Data Set

In [ ]:
from pandas.util import hash_pandas_object
rhc_vereinall_df_hashes = hash_pandas_object(rhc_vereinall_df)

In [ ]:
rhc_vereinall_df_hashes

## Value should be -638227857625026837


In [ ]:
rhc_vereinall_df_hashes.sum()